In [ ]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Activation
from tensorflow.keras.optimizers.experimental import RMSprop

In [ ]:
# Através dessa função o TensorFlow é capaz de baixar o texto diretamente do link
#filepath = tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
filepath = tf.keras.utils.get_file('bible.txt','https://www.gutenberg.org/files/62383/62383-0.txt')


5162002/5162002 [==============================] - 0s 0us/step


In [ ]:
# Tornar as palavras minuscuras e mudar o encoding para utf-8, ajuda na acuária do modelo
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

In [ ]:
#text.title()

# Separar o conteúdo dos dataset
Iremos utilizar somente uma parte dos textos para treinar.

Quanto maior os dados de entrada mais preciso e fidedigno o modelo.
Entretanto maior o tempo e consumo computacional.

O  problema do consumo computacional é diminuido devido o colab, entretnto o tempo consitnua sendo um problema.

In [ ]:
text = text[300000:800000]

# Atribuir valores aos caracteres

Os modelos de ML apenas entendem números como entrada. Então uma das alternativas ao se trabalhar com NLP é dar valores númericos aos characteres.

Essa etapa fica ainda mais fácil ao normalizamos o texto em minisculas e na utf-8.

In [ ]:
characters = sorted(set(text))
#print(characters)

In [ ]:
char_to_index=dict((c,i) for i,c in enumerate(characters))
#print(char_to_index)

In [ ]:
index_to_char=dict((i,c) for i,c in enumerate(characters))
#print(index_to_char)

# Previsão de frases

Consiste em prever o proximo caracters através dos carcteres anteriores. Lembrou-me Bayes....



In [ ]:
#Parametos necessários

# Tamanhos máximo das frases
SEQ_LENGTH= 40

#Tamanhos do conjuntos de caracteres a serem analisados por vez
STEP_SIZE = 3

sentences=[]
next_characters=[]

In [ ]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
  sentences.append(text[i : i+SEQ_LENGTH])
  next_characters.append(text[i+SEQ_LENGTH])

# Tranformando os dados gerados em um vetor númerico

In [ ]:
# forma uma matriz de zeros informando se tal caractere está na frase ou não, con true ou false.
# Essas matrizes serão nossas variares preditoras e resposta
x = np.zeros((len(sentences),SEQ_LENGTH, len(characters)), dtype=np.bool)
y = np.zeros((len(sentences), len(characters)), dtype=np.bool)

<ipython-input-70-ad36a7901484>:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences),SEQ_LENGTH, len(characters)), dtype=np.bool)
<ipython-input-70-ad36a7901484>:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(characters)), dtype=np.bool)


In [ ]:
#print(x)
#print(y)


[[[False  True False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 ...

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False Fal

numpy.ndarray

In [ ]:
#Substituindo os valores da martriz pelos caracterstes correspondentes
for i, sentence in enumerate(sentences):
  for t, character in enumerate(sentence):
    x[i,t, char_to_index[character]] = 1
  y[i, char_to_index[next_characters[i]]] =1


# Criando as Redes Neurais

In [ ]:
model = Sequential()
model.add(LSTM(120,input_shape=(SEQ_LENGTH,len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))



In [ ]:
from keras.engine.training import optimizer
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))
model.fit(x,y, batch_size=254, epochs=4)

Epoch 1/4
657/657 [==============================] - 72s 108ms/step - loss: 1.9364
Epoch 2/4
657/657 [==============================] - 70s 107ms/step - loss: 1.4290
Epoch 3/4
657/657 [==============================] - 71s 109ms/step - loss: 1.2899
Epoch 4/4
657/657 [==============================] - 70s 107ms/step - loss: 1.2175


In [ ]:
model.save('textgenerator.model')

In [ ]:
model2=tf.keras.models.load_model('textgenerator.model')

In [ ]:
# Helper Functions, fornecida pela propria equipe do Keras.
#Ela seleciona a melhor probabilidade
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Função que irá gerar o texto

In [ ]:
def generate_text(length, temperature):
  start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
  generated =''
  sentence = text[start_index: start_index + SEQ_LENGTH]
  generated += sentence
  for i in range(length):
    x = np.zeros((1, SEQ_LENGTH, len(characters)))
    for t,character in enumerate(sentence):
      x[0,t,char_to_index[character]] = 1

    predictions = model.predict(x,verbose=0)[0]
    next_index= sample(predictions,temperature)

    next_character = index_to_char[next_index]

    generated += next_character
    sentence = sentence[1:] + next_character

  return generated


In [ ]:
print('------- Temperature: 0.2 ------------')
print(generate_text(300,0.2))
print('------- Temperature: 0.4 --------------- ')
print(generate_text(300,0.4))
print(' --------------- Temperature: 0.8 ---------------')
print(generate_text(300,0.8))
print('--------------- Temperature: 1 ---------------')
print(generate_text(300,1))

------- Temperature: 0.2 ------------
us: estatuto perpetuo
_é_ por vossas gerações, e o sacerdote o sacerdote o sacerdote o sacerdote o sacerdote o sacerdote o sacerdote o sacerdote o senhor.

22 e a terra do sacerdote o sacerdote o sacerdote o senhor.

22 e o sacerdote o sacerdote o sacerdote o senhor.

22 e a sua mão será sacerdote, e a sua mão será sacerdote, e o s
------- Temperature: 0.4 --------------- 
s _que foram_ contados d’elles, _foram_ a algum alter de um carneiro.

2 e a sua mão sanctos de o sacerdote o sacerdote o sacerdote, [23] a sua mão de moysés, assim o sacerdote o senhor.

2 esta famina de deus povo.

31 e não filho de deus animas, que o sangue do senhor.

23 e disse a sua carne até não os carneiros de deus.

22 
 --------------- Temperature: 0.8 ---------------
ue lhes não tomassemos: sessenta cidades;

23 e o sabba _para_ altados da expiação.

23 fallada mão da mão será omará oquente, na bura incentos, e amaldiça;

22 gerarás a porão a tribu de dazer.

26 astita